In [35]:
import robin_stocks as r
from datetime import datetime
from dateutil import tz
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
import plotly.io as pio
pio.renderers.default='notebook'

from_zone = tz.tzutc()
to_zone = tz.tzlocal()
def utcToLocal(strDate):
    utc = datetime.strptime(strDate, '%Y-%m-%dT%H:%M:%SZ')
    utc = utc.replace(tzinfo=from_zone)
    return utc.astimezone(to_zone)

import configparser
config = configparser.RawConfigParser()
configFilePath = '/Users/philipmassey/.tokens/robinhood.cfg'
config.read(configFilePath)
rhuser = config.get('login', 'user')
rhpwd = config.get('login', 'pwd')
login = r.login(rhuser,rhpwd)

def diffMaxHighLow(df):
    df[['high_price','low_price']] = df[['high_price','low_price']].apply(pd.to_numeric)
    high_price = max(df.high_price)
    low_price = min(df.low_price)
    diff = round((high_price - low_price),2)
    percent = round(diff/low_price,2)
    print('Max-Min:{}, Percent:{:.0%}'.format(diff,percent))

def plotStrikePriceforExpiration(strike_price,optionType,span):
    df = pd.DataFrame(r.get_option_historicals(symbol,expirationDate,strike_price,optionType,span)['data_points'])
    local = utcToLocal(df.iloc[-1:].begins_at.values[-1])
    title = '{} Expiration:{}, Strike: {}, Time:{:%Y-%m-%d %H:%M}'.format(symbol,expirationDate,strike_price,local)
    #df[['close_price', 'high_price','low_price', 'open_price','volume']] = df[['close_price', 'high_price','low_price', 'open_price','volume']].apply(pd.to_numeric)
    #df[['begins_at']] = df[['begins_at']].apply(pd.to_datetime)
    print()
    diffMaxHighLow(df)
    fig = px.line(x=df.begins_at, y=df.high_price)
    fig.update_layout(
        title=title,
        xaxis_title="Date",
        yaxis_title="Option price",
        font=dict(
            family="Courier New, monospace",
            size=18,
            color="#7f7f7f"
        ))
    fig.show()
    
def plotStockPriceforExpiration(strike_price,optionType,span):
    df = pd.DataFrame(r.get_option_historicals(symbol,expirationDate,strike_price,optionType,span)['data_points'])
    dfstock = pd.DataFrame(r.get_historicals(symbol,span=span,bounds='regular'))     
    df['stock_price'] = dfstock['close_price']
    df[['close_price','stock_price']] = df[['close_price','stock_price']].apply(pd.to_numeric)
    df['change_price'] = df['close_price'] - df['close_price'][0:1].values[0]
    local = utcToLocal(df.iloc[-1:].begins_at.values[-1])
    title = '{} Expiration:{}, Strike: {}, Time:{:%Y-%m-%d %H:%M}'.format(symbol,expirationDate,strike_price,local)
    print()
    diffMaxHighLow(df)
    fig = go.Figure()
    fig.add_trace(go.Scatter(x=df.begins_at, y=df.change_price,
                    mode='lines',
                    name='stock_price'))
    # Edit the layout
    fig.update_layout(title=title,
                   xaxis_title='Date',
                   yaxis_title='Stock Price')


    fig.show()

In [20]:
df = pd.DataFrame(r.get_option_historicals(symbol,expirationDate,strike_price,optionType,span)['data_points']) 
dfstock = pd.DataFrame(r.get_historicals(symbol,span=span,bounds='regular'))     
  

*.............................................

In [34]:
#dfstock[['close_price']] = dfstock[['close_price']].apply(pd.to_numeric)
dfstock['change_price'] = dfstock.close_price - dfstock.close_price[0:1].values[0]
dfstock

,begins_at,close_price,high_price,interpolated,low_price,open_price,session,volume,change_price
0,2020-05-22T13:30:00Z,294.1650,294.850000,False,293.910000,294.570000,reg,504880,0.0000
1,2020-05-22T13:35:00Z,293.4600,294.150000,False,293.400000,294.150000,reg,444278,-0.7050
2,2020-05-22T13:40:00Z,293.6100,293.840000,False,293.420000,293.460000,reg,413372,-0.5550
3,2020-05-22T13:45:00Z,294.1000,294.170000,False,293.460000,293.590000,reg,365673,-0.0650
4,2020-05-22T13:50:00Z,294.2150,294.249400,False,293.780000,294.110000,reg,309813,0.0500
5,2020-05-22T13:55:00Z,294.2800,294.410000,False,294.100000,294.230000,reg,285199,0.1150
6,2020-05-22T14:00:00Z,294.0800,294.370000,False,293.830000,294.290000,reg,283284,-0.0850
7,2020-05-22T14:05:00Z,293.7100,294.170000,False,293.560000,294.100000,reg,345458,-0.4550
8,2020-05-22T14:10:00Z,293.3550,293.910000,False,293.320000,293.703000,reg,330149,-0.8100
9,2020-05-22T14:15:00Z,293.6996,293.820000,False,293.220000,293.350000,reg,253656,-0.4654


### Expiration Dates

In [16]:
symbol = 'SPY'
expiration_dates = r.get_chains(symbol)['expiration_dates']
print(expiration_dates)

['2020-05-27', '2020-05-29', '2020-06-01', '2020-06-03', '2020-06-05', '2020-06-08', '2020-06-10', '2020-06-12', '2020-06-15', '2020-06-17', '2020-06-19', '2020-06-22', '2020-06-24', '2020-06-26', '2020-06-29', '2020-06-30', '2020-07-01', '2020-07-02', '2020-07-17', '2020-08-21', '2020-09-18', '2020-09-30', '2020-10-16', '2020-11-20', '2020-12-18', '2020-12-31', '2021-01-15', '2021-03-19', '2021-03-31', '2021-06-18', '2021-09-17', '2021-12-17', '2022-01-21', '2022-03-18', '2022-12-16']


### Strike Prices for Expiration Date

In [17]:
expirationDate = expiration_dates[0]
volume_limit = 0
optionData = r.find_options_for_list_of_stocks_by_expiration_date([symbol], expirationDate=expirationDate,optionType='call') 
dfoptions = pd.DataFrame((filter(lambda x:x['volume']>volume_limit,optionData)))
#dfoptions[['strike_price','break_even_price','volume','last_trade_price', 'previous_close_price', 'last_trade_size','open_interest']].sort_values(by='strike_price')
#dfoptions[['strike_price','break_even_price','volume','last_trade_price', 'previous_close_price', 'last_trade_size','open_interest']].sort_values(by='volume',ascending=False)
dfstrike_prices = dfoptions.sort_values(by='volume',ascending=False)[0:5][['strike_price','volume']]
dfstrike_prices

*.............................................

,strike_price,volume
38,296.0000,10397
67,300.0000,9259
66,295.0000,8994
26,302.0000,6668
39,294.0000,3663


In [37]:
optionType = 'call'
span = 'day'
strike_price = dfstrike_prices.iloc[0].strike_price
plotStrikePriceforExpiration(strike_price,optionType,span)
plotStockPriceforExpiration(strike_price,optionType,span)

*.............................................
Max-Min:0.62, Percent:38%


*.............................................
Max-Min:0.62, Percent:38%


In [38]:
optionType = 'call'
span = 'week'
strike_price = dfstrike_prices.iloc[0].strike_price
plotStrikePriceforExpiration(strike_price,optionType,span)
plotStockPriceforExpiration(strike_price,optionType,span)

*.............................................
Max-Min:3.03, Percent:187%


*.............................................
Max-Min:3.03, Percent:187%


In [39]:
span = 'year'
strike_price = dfstrike_prices.iloc[1].strike_price
plotStrikePriceforExpiration(strike_price,optionType,span)
plotStockPriceforExpiration(strike_price,optionType,span)

*.............................................
Max-Min:6.01, Percent:60100%


*.............................................
Max-Min:6.01, Percent:60100%


### Stock Price

In [47]:
span='week'

plotStockPriceforExpiration(strike_price,optionType,span)

*.
Max-Min:0.27, Percent:2700%
